# Civitai SDK Text2Img Example
This notebook demonstrates how to generate an image using the Civitai SDK.

In [1]:
%pip install civitai-py
%pip install ipython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 57.3 MB/s eta 0:00:00


In [2]:
import os

api_token = "a7f8406491e96317f62b373997d0cc28" #input("Enter your Civitai API token: ")
os.environ['CIVITAI_API_TOKEN'] = api_token

### Example 1: Manually polling for status of job

In [6]:
# Import the Civitai SDK
import civitai

# Define the input parameters for the image creation
input = {
    "model": "urn:air:sdxl:checkpoint:civitai:101055@128078",
    "params": {
        "prompt": "RAW photo, face portrait photo of woman, wearing black dress, happy face, hard shadows, cinematic shot, dramatic lighting",
        "negativePrompt": "(deformed, distorted, disfigured:1.3)",
        "scheduler": "EulerA",
        "steps": 20,
        "cfgScale": 7,
        "width": 768,
        "height": 512,
        "seed": -1,
        "clipSkip": 1
    },
}

# Generate the image
# We're using await in the notebook to handle the coroutine
response = civitai.image.create(input)
print("Response:", response)

Response: <Task pending name='Task-18' coro=<Civitai.Image.create.<locals>.async_create() running at /usr/local/lib/python3.12/dist-packages/civitai/__init__.py:43>>


You can then retrieve the job status and image using the job token or job ID.

In [8]:
from IPython.display import Image, display

print(response['result'])

job_token = response['token']
job_id = response['jobs'][0]['jobId']

# Retrieve job status and image
response = civitai.jobs.get(token=job_token, job_id=job_id)
print("Job Status Response:", response)

if response['jobs'][0]['result'].get('available'):
    image_url = response['jobs'][0]['result'].get('blobUrl')
    if image_url:
        display(Image(url=image_url))
    else:
        print("Image URL not found in the job result.")
else:
    print("No image was created, the job is not yet complete, or the result is not available.")

<Task finished name='Task-18' coro=<Civitai.Image.create.<locals>.async_create() done, defined at /usr/local/lib/python3.12/dist-packages/civitai/__init__.py:43> result={'jobs': [{'cost': 0.9600000000000002, 'jobId': '3c62e87b-e8e...-ce1fc4dcfa49', 'result': [{'available': False, 'blobKey': '46F6DD983893...33BA8C152325D', 'seed': 285377914}], 'scheduled': True}], 'token': 'eyJKb2JzIjpb...jZmE0OSJdfQ=='}>


TypeError: '_asyncio.Task' object is not subscriptable

### Example 2: Run image generation in the background.

By adding `wait=True` in the `civitai.image.create` call, this runs the text2img job in the background and only returns a response after job completion or timeout. It has a default timeout of 5 minutes.


In [5]:
# Generate the image and wait for completion
response = await civitai.image.create(input, wait=True)
print("Response:", response)

if response['jobs'][0]['result'].get('available'):
    image_url = response['jobs'][0]['result'].get('blobUrl')
    if image_url:
        display(Image(url=image_url))
    else:
        print("Image URL not found in the job result.")
else:
    print("No image was created, the job is not yet complete, or the result is not available.")

AttributeError: 'list' object has no attribute 'get'